# 5. web scraping workbook
Purpose: To demonstrate web scraping abilities since no web scraping was done for Project 2. Code can be used to import tables from linked pages that share a common source page. Example is to pull tables of the list of hospitals in each state (found on separate pages) from wikipedia's source page for list of all hospitals by state. Code assumes the table of interest is the first table. Code saves tables as pickles.

Option 1: pull tables using HTML reference<br>
Option 2: pull tables using pandas

## Set up

In [1]:
from __future__ import print_function, division

# if needed: pip install requests or conda install requests
import requests

requests.__path__

from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time

import chromedriver_binary

import pandas as pd
import pickle

# Access and pull data

In [191]:
main_url = 'https://en.wikipedia.org/wiki/Lists_of_hospitals_in_the_United_States'

In [ ]:
#List of states of interest
state_list = ['Alabama','Connecticut','Idaho']

### Option 1: Pull tables using HTML reference

In [192]:
driver = webdriver.Chrome()
driver.get(main_url)

In [ ]:
for state in state_list:
    #click on list
    driver.find_element_by_link_text(state).click()
    #pull table info into list using beautiful soup
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    #get table items
    tables = soup.find_all('table')
    rows = [row for row in tables[0].find_all('tr')]
    headers = [x.getText().strip() for x in tables[0].find_all('th')]
    
    #pull table data into dataframe
    for i in range(len(headers)): 
    #first column
        if i == 0:
            df = []
            for row in range(1,len(rows)):
                df.append(rows[row].find_all('td')[i].text.strip())
            df = pd.DataFrame(df)
        else:
            new_col = []
            for row in range(1,len(rows)):
                new_col.append(rows[row].find_all('td')[i].text.strip())
            df[i] = new_col
    
    #save out dataframe
    headers_dict = dict(list(zip(range(len(headers)),headers)))
    df.rename(columns=headers_dict,inplace=True)
    df['State'] = state 
    
    with open('web scraping/'+state+'.pickle', 'wb') as to_write:
        pickle.dump(df, to_write)
    
    #return to main page
    driver.back()

In [190]:
driver.close()

### Option 2: Pull tables using Pandas

In [ ]:
driver = webdriver.Chrome()
driver.get(main_url)

In [212]:
for state in state_list:
    #click on list
    driver.find_element_by_link_text(state).click()

    tables = pd.read_html(driver.current_url)
    df = tables[0]
    df['State'] = state 
    
    with open('web scraping/'+state+'.pickle', 'wb') as to_write:
        pickle.dump(df, to_write)
    
    #return to main page
    driver.back()

In [ ]:
driver.close()